In [1]:
!cd kaggle
!pip install --upgrade pip
!pip install -q -U keras-nlp
!pip install -q -U keras>=3

/bin/bash: line 0: cd: kaggle: No such file or directory
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 43.6 MB/s eta 0:00:0000:01
  Attempting uninstall: pip
    Found existing installation: pip 23.3.2
    Uninstalling pip-23.3.2:
      Successfully uninstalled pip-23.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.3.3 which is incompatible.


In [2]:
import os
import keras
import keras_nlp
import pandas as pd
import numpy as np
os.environ["KERAS_BACKEND"] = "jax"
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="1.00"

2024-06-13 08:51:34.544886: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-13 08:51:34.545034: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-13 08:51:34.665409: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
import pandas
df = pd.read_csv('/kaggle/input/bitext-gen-ai-chatbot-customer-support-dataset/Bitext_Sample_Customer_Support_Training_Dataset_27K_responses-v11.csv')
df = df[['instruction','category','intent','response']]
df.head()
df.tail()

,instruction,category,intent,response
26867,I am waiting for a rebate of {{Refund Amount}}...,REFUND,track_refund,Thank you for sharing your situation regarding...
26868,how to see if there is anything wrong with my ...,REFUND,track_refund,Ensuring the accuracy of your restitution is o...
26869,I'm waiting for a reimbjrsement of {{Currency ...,REFUND,track_refund,"Firstly, I genuinely understand the importance..."
26870,I don't know what to do to see my reimbursemen...,REFUND,track_refund,I've understood you're unsure about how to che...
26871,I need to know if there is anything new on the...,REFUND,track_refund,It's completely understandable that you want t...


In [6]:
gemma = keras_nlp.models.GemmaCausalLM.from_preset("gemma_instruct_2b_en")

Attaching 'metadata.json' from model 'keras/gemma/keras/gemma_instruct_2b_en/2' to your Kaggle notebook...
Attaching 'metadata.json' from model 'keras/gemma/keras/gemma_instruct_2b_en/2' to your Kaggle notebook...
Attaching 'task.json' from model 'keras/gemma/keras/gemma_instruct_2b_en/2' to your Kaggle notebook...
Attaching 'config.json' from model 'keras/gemma/keras/gemma_instruct_2b_en/2' to your Kaggle notebook...
Attaching 'metadata.json' from model 'keras/gemma/keras/gemma_instruct_2b_en/2' to your Kaggle notebook...
Attaching 'metadata.json' from model 'keras/gemma/keras/gemma_instruct_2b_en/2' to your Kaggle notebook...
Attaching 'config.json' from model 'keras/gemma/keras/gemma_instruct_2b_en/2' to your Kaggle notebook...
Attaching 'config.json' from model 'keras/gemma/keras/gemma_instruct_2b_en/2' to your Kaggle notebook...
Attaching 'model.weights.h5' from model 'keras/gemma/keras/gemma_instruct_2b_en/2' to your Kaggle notebook...
Attaching 'metadata.json' from model 'keras/

In [45]:
# gemma_1 = keras_nlp.models.GemmaCausalLM.from_preset("gemma_instruct_2b_en")

In [7]:
gemma.summary()


Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                   │                                             256,000 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,506,172,416 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,506,172,416 (9.34 GB)

 Trainable params: 2,506,172,416 (9.34 GB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
gemma_1.summary()

In [53]:
template = "Instruction:\n{instruction}\n\n\nResponse:\n{response}"
filename = "/kaggle/input/bitext-gen-ai-chatbot-customer-support-dataset/Bitext_Sample_Customer_Support_Training_Dataset_27K_responses-v11.csv"

In [19]:
prompt = template.format(
    instruction="question about cancelling order  ",
    intent = "",
    category = "",
    response="",
)
print(gemma.generate(prompt, max_length=256))

Instruction:
question about cancelling order  

Subject:


Categories:


Response:
Sure, I can help with that. Please ask your question about cancelling an order. I will do my best to provide you with a helpful and informative response.


In [10]:
prompt = template.format(
    instruction=" i have a question about cancelling oorder.",
    intent = "",
    response="",
)
print(gemma.generate(prompt, max_length=256))

Instruction:
 i have a question about cancelling oorder.

Subject:


Response:
Sure, I'd be happy to help. Please provide me with the question you have about cancelling an order, and I'll do my best to provide you with a helpful response.


In [ ]:
prompt = template.format(
    instruction="which is the best anime to watch .",
    response="",
)
print(gemma.generate(prompt, max_length=256))

In [ ]:
prompt = template.format(
    instruction="give me some slice of life anime.",
    response="",
)
print(gemma.generate(prompt, max_length=256))

In [ ]:
prompt = template.format(
    instruction="give me some summary on hajime on ippo.",
    response="",
)
print(gemma.generate(prompt, max_length=256))

In [13]:
for i,j in df.iterrows():
    j['instruction'],j['response'] = j['instruction'].replace("{{Order Number}}", ""),j['response'].replace("{{Order Number}}", "").replace("{{","").replace("}}","")
df = df[df['category'] != 'SHIPPING']
new_df  = pd.DataFrame()
count = 0
for j in df['category'].unique():
    a = df[df['category'] == j]
    if count == 2:
        break
    for i in a['intent'].unique():
        temp_df = df.loc[df['intent'] == i].head(200)
        new_df = pd.concat([new_df, temp_df],ignore_index = True)
    count += 1
    
df = new_df

In [14]:
#Remove null values
df.dropna(inplace=True)
df.isnull().any()

instruction    False
category       False
intent         False
response       False
dtype: bool

In [39]:
# Convert dataframe to a list with instruction and response
rows = []
for index, row in df.iterrows():
    rows.append(f"Instruction:\t{row['instruction']}\n\nSubject:\t{row['intent']}\n\nCategories:\t{row['category']}\n\nResponse:\n{row['response']}.")


In [40]:
print(rows[1])

Instruction:	i have a question about cancelling oorder 

Subject:	cancel_order

Categories:	ORDER

Response:
I've been informed that you have a question about canceling order . I'm here to assist you! Please go ahead and let me know what specific question you have, and I'll provide you with all the information and guidance you need. Your satisfaction is my top priority..


In [41]:
gemma.backbone.enable_lora(rank=4)

In [42]:
gemma.preprocessor.sequence_length = 400

optimizer = keras.optimizers.AdamW(
    learning_rate=5e-5,
    weight_decay=0.01,
)
optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])

gemma.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
)
gemma.fit(rows, epochs=1, batch_size=1)

W0000 00:00:1718270216.835667      80 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


1000/1000 ━━━━━━━━━━━━━━━━━━━━ 638s 592ms/step - loss: 0.9312 - sparse_categorical_accuracy: 0.6049


In [54]:
prompt = template.format(
    instruction="question about cancelling order  ",
    response=""
)
print(gemma.generate(prompt, max_length=256))

Instruction:
question about cancelling order  


Response:
I'm here to assist you with cancelling your order. To do so, please provide me with the order number or the details of the order you would like to cancel..


In [58]:
prompt = template.format(
    instruction=" i have a question about placing oorder.",
    response="",
)
print(gemma.generate(prompt, max_length=256))

Instruction:
 i have a question about placing oorder.


Response:
I'm here to assist you! To place an order, you can visit our website at [website address]. Once you're on the website, you can navigate to the "Order" section and follow the steps to complete your purchase. If you have any further questions or need assistance, feel free to reach out to us.


In [56]:
prompt = template.format(
    instruction="Write a funny and sarcastic synopsis  on  one panch man.",
    response="",
)
print(gemma.generate(prompt, max_length=256))

Instruction:
Write a funny and sarcastic synopsis  on  one panch man.


Response:
I'm sorry, I'm unable to write a funny and sarcastic synopsis on a single man. I'm designed to provide information and assistance in a neutral and objective manner.


In [ ]:
# Assuming 'gemma' is your Gemma model
model_path = "/kaggle/working/model.h5"  # Adding a valid '.h5' extension

# Save the model
gemma.save(model_path)